In [46]:
PROJECT_ID = "wb-ai-acltr-tbs-3-pr-a62583"
PROJECT_REGION = "northamerica-northeast1"
BQ_DATASET_NAME = "b2b_wf_prediction"
BQ_FORECAST_TABLE= "bq_wf_forecast"

In [47]:
TIMESTAMP = '2025-02-12 11:11:20.260889'
MODEL = 'AutoML'

In [48]:
import sys
sys.path.insert(0, '/workspaces/b2b-wf-experiments/src')

from components.data_evaluation_preprocessor import DataEvaluationPreprocessor
from components.data_evaluator import Evaluation

In [49]:
import pandas as pd
import numpy as np

In [50]:
from google.cloud import bigquery 

client = bigquery.Client(
    project=PROJECT_ID,
    location=PROJECT_REGION
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [65]:
forecast_query = f"""
SELECT
  DATE_TRUNC(Appointment_Day, MONTH) AS Appointment_Day,
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type,
  SUM(SWT) as SWT
FROM `{BQ_DATASET_NAME}.bq_wf_forecast`
WHERE 
  Model = 'AutoML'
  AND Forecast_Date = CAST('{TIMESTAMP}' AS TIMESTAMP)
GROUP BY
  DATE_TRUNC(Appointment_Day, MONTH),
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type
"""
historical_query = f"""
SELECT
  DATE_TRUNC(Appointment_Day, MONTH) AS Appointment_Day,
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type,
  SUM(SWT) as SWT
FROM `{BQ_DATASET_NAME}.vw_wf_historical`
GROUP BY
  DATE_TRUNC(Appointment_Day, MONTH),
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type
"""

In [52]:
import pandas as pd

def compute_series_stats(df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:

    df["Appointment_Day"] = pd.to_datetime(df["Appointment_Day"])
    
    mask = (df["Appointment_Day"] >= pd.to_datetime(start_date)) & (df["Appointment_Day"] <= pd.to_datetime(end_date))
    df_filtered = df.loc[mask].copy()
    
    #grouping_cols = ["District", "Technology", "Work_Order_Action", "Product", "Work_Force"]
    grouping_cols = ["District","Work_Order_Action_Grp", "Product_Grp"]
    
    stats_df = df_filtered.groupby(grouping_cols)["SWT"].agg(["mean", "std"]).reset_index()
    stats_df["Series"] = stats_df[grouping_cols].astype(str).agg(" | ".join, axis=1)
    stats_df = stats_df[["Series", "mean", "std"]]
    
    return stats_df
import pandas as pd

import pandas as pd

def override_odd_predictions(hist_df: pd.DataFrame, 
                             pred_df: pd.DataFrame, 
                             multiplier: float = 2.0) -> pd.DataFrame:
    merged_df = pd.merge(hist_df, pred_df, on="Series", suffixes=("_hist", "_pred"))
    
    lower_bound = merged_df["mean_hist"] - multiplier * merged_df["std_hist"]
    upper_bound = merged_df["mean_hist"] + multiplier * merged_df["std_hist"]
    
    merged_df["adjusted_pred_mean"] = merged_df.apply(
        lambda row: np.nan if lower_bound[row.name] <= row["mean_pred"] <= upper_bound[row.name]
                    else row["mean_hist"],
        axis=1
    )
    
    return merged_df[["Series", "adjusted_pred_mean"]]

import pandas as pd

def adjust_swt_values(df_data: pd.DataFrame, df_adjust: pd.DataFrame, 
                      grouping_cols: list[str] = None, separator: str = " | ") -> pd.DataFrame:
    if grouping_cols is None:
        grouping_cols = ["District", "Work_Order_Action_Grp", "Product_Grp"]

    df_data = df_data.copy() 
    df_data["Series"] = df_data[grouping_cols].astype(str).agg(separator.join, axis=1)

    df_merged = pd.merge(df_data, df_adjust[["Series", "adjusted_pred_mean"]],
                         on="Series", how="left")

    df_merged["SWT"] = df_merged.apply(
        lambda row: row["adjusted_pred_mean"] if pd.notna(row["adjusted_pred_mean"]) else row["SWT"],
        axis=1
    )
    
    return df_merged[["Appointment_Day", "Product_Grp", "Work_Order_Action_Grp", "District", "Region_Type", "SWT"]]

import pandas as pd

def group_swt_by_columns(df: pd.DataFrame, group_cols: list[str] = None) -> pd.DataFrame:
    if group_cols is None:
        group_cols = ['Appointment_Day', 'Product_Grp', 'Work_Order_Action_Grp', 'District', 'Region_Type']
    
    grouped_df = df.groupby(group_cols, as_index=False)['SWT'].sum()
    
    return grouped_df

In [53]:
#forecast_df = pd.read_csv('forecasts.csv', index_col=False)
forecast_df = client.query_and_wait(forecast_query).to_dataframe()
forecast_df.head()

,Appointment_Day,Product_Grp,Work_Order_Action_Grp,District,Region_Type,SWT
0,2024-10-01,Managed,Install,AFF Abitibi,Tier 2,16.498057
1,2024-11-01,Managed,Install,AFF Abitibi,Tier 2,16.297452
2,2024-08-01,Managed,MAC & Out,AFF Abitibi,Tier 2,8.686646
3,2024-10-01,Managed,MAC & Out,AFF Abitibi,Tier 2,8.256686
4,2024-09-01,Managed,Repair,AFF Abitibi,Tier 2,8.999448


In [66]:
#historical_df = pd.read_csv('historical.csv', index_col=False)
historical_df = client.query_and_wait(historical_query).to_dataframe()
historical_df.head()

,Appointment_Day,Product_Grp,Work_Order_Action_Grp,District,Region_Type,SWT
0,2022-03-01,Managed,Install,AFF Grand-Nord,Tier 2,3.000000000
1,2022-03-01,Managed,MAC & Out,AFF Saguenay,Tier 3,21.000000000
2,2022-03-01,Managed,Install,AFF Rimouski,Tier 3,108.000000000
3,2022-03-01,Managed,MAC & Out,BUS SW Peninsula ON,None,31.000000000
4,2022-03-01,Managed,MAC & Out,BUS Sarnia,None,4.000000000


In [ ]:
fit_data = historical_df[historical_df['Appointment_Day'].isin(pd.date_range(start='2022-01-01', end='2024-06-01'))]

In [55]:
forecast_means = compute_series_stats(forecast_df, '2024-07-01', '2025-01-01')
historical_means = compute_series_stats(historical_df, '2024-01-01', '2024-06-01')

In [56]:
adjustements = override_odd_predictions(historical_means, forecast_means, multiplier=1.5)
adjusted_df = adjust_swt_values(forecast_df, adjustements)

In [57]:
adjusted_df.head()

,Appointment_Day,Product_Grp,Work_Order_Action_Grp,District,Region_Type,SWT
0,2024-10-01,Managed,Install,AFF Abitibi,Tier 2,10.500000
1,2024-11-01,Managed,Install,AFF Abitibi,Tier 2,10.500000
2,2024-08-01,Managed,MAC & Out,AFF Abitibi,Tier 2,4.166667
3,2024-10-01,Managed,MAC & Out,AFF Abitibi,Tier 2,4.166667
4,2024-09-01,Managed,Repair,AFF Abitibi,Tier 2,2.000000


In [58]:
ground_truth = historical_df[historical_df['Appointment_Day'].isin(pd.date_range(start='2024-07-01', end='2025-01-01'))]
ground_truth.head()

,Appointment_Day,Product_Grp,Work_Order_Action_Grp,District,Region_Type,SWT
825,2024-08-01,Managed,Install,BUS Calgary South West,Tier 1,32.500000000
826,2024-08-01,Unmanaged,Install,BUS Calgary South East,Tier 1,301.500000000
827,2024-08-01,Unmanaged,Install,BUS Kootenay Cranbrook,Tier 3,79.000000000
828,2024-08-01,Managed,Install,AFF Montmagny,Tier 2,6.000000000
829,2024-08-01,Managed,Install,AFF Carleton,Tier 3,7.000000000


In [62]:
historical_data = DataEvaluationPreprocessor(ground_truth)
forecast_data = DataEvaluationPreprocessor(forecast_df)

In [63]:
evaluation = Evaluation(historical_data, forecast_data)

minimal_features_rmse = {
    'overall': evaluation.calculate_metric('rmse'),
    'Tier 1': evaluation.calculate_metric('rmse', filters={'Region_Type': 'Tier 1'}),
    'Tier 2': evaluation.calculate_metric('rmse', filters={'Region_Type': 'Tier 2'}),
    'Tier 3': evaluation.calculate_metric('rmse', filters={'Region_Type': 'Tier 3'})
}

minimal_features_wape = {
    'overall': evaluation.calculate_metric('wape'),
    'Tier 1': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 1'}),
    'Tier 2': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 2'}),
    'Tier 3': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 3'})
}

In [64]:
minimal_features_wape

{'overall': 0.4343063132964376,
 'Tier 1': 0.36882470529305883,
 'Tier 2': 0.4615778625080444,
 'Tier 3': 0.9800519205852429}

In [67]:
excel_model = {'overall': 0.243,
 'Tier 1': 0.191,
 'Tier 2': 0.286,
 'Tier 3': 0.546}

daily_minimal_post_process = {'overall': 0.297,
 'Tier 1': 0.231,
 'Tier 2': 0.361,
 'Tier 3': 0.672
}

daily_minimal = {'overall': 0.434,
 'Tier 1': 0.368,
 'Tier 2': 0.461,
 'Tier 3': 0.980}


monthly_panorama = {'overall': 0.322,
 'Tier 1': 0.287,
 'Tier 2': 0.338,
 'Tier 3': 0.606
 }

In [72]:
import plotly.express as px
px.defaults.template = "plotly_dark"
def plot_comparison(metric, list_of_dicts, model_labels):
    """
    Given multiple dictionaries of metrics and a corresponding list of labels,
    this function plots a grouped bar chart comparing each metric across models.
    
    list_of_dicts: list of dictionaries (e.g., each entry is { 'overall_rmse': ..., 'Tier1_rmse': ..., ... })
    model_labels: list of model names (e.g., ['Excel Model', 'AutoML Model', 'Another Model'])
    """
    # Build a rows list of dicts: { "model": label, "metric": metric_key, "value": metric_value }
    rows = []
    for model_name, scores_dict in zip(model_labels, list_of_dicts):
        for metric_name, metric_value in scores_dict.items():
            rows.append({
                "model": model_name,
                "metric": metric_name,
                "value": metric_value
            })
    
    df = pd.DataFrame(rows)
    
    # Create a grouped bar chart
    fig = px.bar(
        df,
        x="metric",
        y="value",
        color="model",
        barmode="group",
        title=f"{metric} comparison (lower is better)"
    )
    fig.update_layout(xaxis_title="Metric", yaxis_title=metric)
    fig.show()

In [73]:
plot_comparison(
    "WAPE",
    [
        excel_model,
        monthly_panorama,
        daily_minimal,
        daily_minimal_post_process
    ],
    [
        "Excel",
        "Montly AutoML (Panorama)",
        "Small Daly AutoML (Panorama)",
        "Small Daily PostProcessed AutoML (Panorama)"
    ]
)